### API 키 불러오기

In [1]:
import os
import configparser

In [2]:
config = configparser.ConfigParser()
config.read('./secrets.ini')

['./secrets.ini']

In [3]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
serper_api_key = config['SERPER']['SERPER_API_KEY']
serp_api_key = config['SERPAPI']['SERPAPI_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})
os.environ.update({'SERPER_API_KEY': serper_api_key})
os.environ.update({'SERPAPI_API_KEY': serp_api_key})

In [4]:
from typing import List, Union
import re
import json

import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate, load_prompt, BaseChatPromptTemplate
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.vectorstores import DocArrayInMemorySearch, Chroma

### Get Stage Analyzer Prompt

In [5]:
stage_analyzer_inception_prompt = load_prompt("./templates/stage_analyzer_inception_prompt_template.json")
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.0)
stage_analyzer_chain = LLMChain(
    llm=llm,
    prompt=stage_analyzer_inception_prompt, 
    verbose=True, 
    output_key="stage_number")

### Load wine database json

In [6]:
df = pd.read_json('./data/unified_wine_data.json', encoding='utf-8', lines=True)

In [7]:
df.head()

,url,site_name,price,name,en_name,img_url,body,acidity,tannin,sweetness,alcohol,wine_type,country,grape,rating,pickup_location,vivino_link
0,https://www.winenara.com/shop/product/product_...,winenara,49000,모노폴 클라시코,Monopole Classico,https://www.winenara.com/uploads/product/550/1...,3,,,-1,-1,화이트,스페인,,3.8,,https://www.vivino.com/monopole-la-rioja-blanc...
1,https://www.winenara.com/shop/product/product_...,winenara,32000,슐럼베르거 로제 스페셜 브뤼,Schlumberger Rose Special Brut,https://www.winenara.com/uploads/product/550/d...,3,,,-1,-1,스파클링,독일,,3.8,,https://www.vivino.com/schlumberger-spring-edi...
2,https://www.winenara.com/shop/product/product_...,winenara,50000,SET)페데럴리스트 샤르도네 원통 패키지,SET)THE FEDERALIST CHARDONNAY,https://www.winenara.com/uploads/product/550/d...,3,,,-1,-1,화이트,미국,,3.7,,https://www.vivino.com/federalist-chardonnay-m...
3,https://www.winenara.com/shop/product/product_...,winenara,55000,베니카 트레 비니스,VENICA TRE VIGNIS,https://www.winenara.com/uploads/product/550/c...,4,,,-1,-1,화이트,이탈리아,,3.9,,https://www.vivino.com/US-CA/en/venica-venica-...
4,https://www.winenara.com/shop/product/product_...,winenara,24900,SET)빌라엠비앙코 + 글라스2개 윈터패키지,SET)VILLA M Bianco + GLASS WINTER PACKAGE,https://www.winenara.com/uploads/product/550/a...,-1,,,4,-1,디저트,이탈리아,,3.9,,https://www.vivino.com/villa-m-bianco/w/1774733


### Prepare Langchain Tool

#### Tool1: Wine database 1

In [8]:
loader =DataFrameLoader(data_frame=df, page_content_column='name')
docs = loader.load()
embeddings = OpenAIEmbeddings()

아래는 wine database1에 metadata_field Attribute이다. 아래를 기준으로 서치를 진행하게 된다.

In [9]:
metadata_field_info = [
    AttributeInfo(
        name="body",
        description="1-5 rating for the body of wine",
        type="int",
    ),
    AttributeInfo(
        name="sweetness",
        description="1-5 rating for the sweetness of wine",
        type="int",
    ),
    AttributeInfo(
        name="alcohol",
        description="1-5 rating for the alcohol of wine",
        type="int",
    ),
    AttributeInfo(
        name="price",
        description="The price of the wine",
        type="int",
    ),
    AttributeInfo(
        name="rating", 
        description="1-5 rating for the wine", 
        type="float"
    ),
    AttributeInfo(
        name="wine_type", 
        description="The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'", 
        type="string"
    ),
    AttributeInfo(
        name="country", 
        description="The country of wine. It can be '기타 신대륙', '기타구대륙', '뉴질랜드', '독일', '미국', '스페인', '아르헨티나', '이탈리아', '칠레', '포루투칼', '프랑스', '호주'", 
        type="float"
    ),
]

In [10]:
vectorstore = Chroma.from_documents(docs, embeddings)
document_content_description = "Database of a wine"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)  # Added missing closing parenthesis


In [11]:
retriever.get_relevant_documents('{"wine_type":"레드", "body": "lt 3 gt 0"}') # gt means greater than, lt means less than, eq means equal to

c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\langchain\chains\llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query=' ' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='wine_type', value='레드'), Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.GT: 'gt'>, attribute='body', value=0), Comparison(comparator=<Comparator.LT: 'lt'>, attribute='body', value=3)])]) limit=None


[Document(page_content='바 독 피노누아', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=03P976', 'site_name': 'winenara', 'price': 29000, 'en_name': 'BAR DOG PINOT NOIR', 'img_url': 'https://www.winenara.com/uploads/product/550/1936_detail_084.png', 'body': 2, 'acidity': '', 'tannin': '', 'sweetness': -1, 'alcohol': -1, 'wine_type': '레드', 'country': '미국', 'grape': '', 'rating': 3.6, 'pickup_location': '', 'vivino_link': 'https://www.vivino.com/US-CA/en/bar-dog-pinot-noir/w/7129723'}),
 Document(page_content='루이라뚜르 피노누아', metadata={'url': 'https://www.winenara.com/shop/product/product_view?product_cd=03H965', 'site_name': 'winenara', 'price': 52000, 'en_name': 'LOUIS LATOUR PINOT NOIR', 'img_url': 'https://www.winenara.com/uploads/product/550/493_detail_025.png', 'body': 2, 'acidity': '', 'tannin': '', 'sweetness': -1, 'alcohol': -1, 'wine_type': '레드', 'country': '프랑스', 'grape': '', 'rating': 3.6, 'pickup_location': '', 'vivino_link': 'https://www.vivino.com/G

#### Tool2: Search specific wine with url

In [12]:
def search_with_url(query):
    return SeleniumURLLoader(urls=[query]).load()

#### Tool3: Wine database 2

In [13]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

#### Tool4: Search in Google

In [14]:
search = SerpAPIWrapper()

In [15]:
tools = [
    Tool(
        name="Wine database",
        func=retriever.get_relevant_documents,
        description="""
Database about the wines in wine store. You can get information such as the price of the wine, purchase URL, features, rating information, and more.
You can search wines with the following attributes:
- body: 1-5 rating int for the body of wine. You have to specify greater than or less than. For example, if you want to search for wines with a body rating of less than 3, enter 'body: gt 0 lt 3'
- price: The price range of the wine. Please enter the price range in the form of range. For example, if you want to search for wines that cost less than 20,000 won, enter 'price: gt 0 lt20000'
- rating: 1-5 rating float for the wine. You have to specify greater than or less than. For example, if you want to search for wines with a rating of less than 3, enter 'rating: gt 0 lt 3'
- wine_type: The type of wine. It can be '레드', '로제', '스파클링', '화이트', '디저트', '주정강화'
- name: The name of wine. 입력할 때는 '와인 이름은 "비냐 조잘" 입니다' 이런 식으로 입력해주세요.
"""
    ),
    Tool(
        name = "Search specific wine with url",
        func=search_with_url,
        description="Search specific wine with url. Query must be url"
    ),
    Tool(
        name = "Wine database 2",
        func=index.query,
        description="Database about the wines in wine store. You can use this tool if you're having trouble getting information from the wine database tool above. Query must be in String"
    ),
    Tool(
        name = "Search",
        func=search.run,
        description="Useful for when you need to ask with search. Search in English only."
    ),
]

In [16]:
template = """
Your role is a chatbot that asks customers questions about wine and makes recommendations.
Never forget your name is "이우선".
Keep your responses in short length to retain the user's attention. 
Only generate one response at a time! When you are done generating, end with '<END_OF_TURN>' to give the user a chance to respond.
Responses should be in Korean.

Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
이우선: the final response to the user

You must respond according to the conversation stage within the triple backticks and conversation history within in '======'.

Current conversation stage: 
```{conversation_stage}```

Conversation history: 
=======
{conversation_history}
=======

Last user saying: {input}
{agent_scratchpad}
"""

conversation_stages_dict = {
    "1": "Start: Start the conversation by introducing yourself. Be polite and respectful while maintaining a professional tone of conversation.",
    "2": "Analyze: Identify the user's preferences in order to make wine recommendations. Ask questions to understand the preferences of your users in order to make wine recommendations. Ask only one question at a time. The wine database tool is not available here.",
    "3": "Recommendation: Recommend the right wine based on the user's preferences identified. Recommendations must be limited to wines in wine database, and you can use tools to do this.",
    "4": "After recommendation: After making a wine recommendation, it asks if the user likes the wine you recommended, and if they do, it provides a link to it. Otherwise, it takes you back to the recommendation stage.",
    "5": "Close: When you're done, say goodbye to the user.",
    "6": "Question and Answering: This is where you answer the user's questions. To answer user question, you can use the search tool or the wine database tool.",
    "7": "Not in the given steps: This step is for when none of the steps between 1 and 6 apply.",
}

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        stage_number = kwargs.pop("stage_number")
        kwargs["conversation_stage"] = conversation_stages_dict[stage_number]
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps", "conversation_history", "stage_number"]
)

In [17]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "이우선: " in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("이우선: ")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

### Define Langchain Agent

In [18]:
llm_chain = LLMChain(llm=ChatOpenAI(model='gpt-4', temperature=0.0), prompt=prompt, verbose=True,)

tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [19]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

### Start Conversation

In [20]:
user_response = ""
conversation_history, pre_conversation_history = """""", """"""
stage_history = ""

In [21]:
user_response = input("User: ")
pre_conversation_history = conversation_history
conversation_history += f"User: {user_response} <END_OF_TURN>\n"

In [22]:
stage_number = stage_analyzer_chain.run({'conversation_history': conversation_history, 'stage_history': stage_history})
print(f'stage_number: {stage_number}')
stage_history += stage_number if stage_history == "" else ", " + stage_number
response = agent_executor.run({'input':user_response, 'conversation_history': pre_conversation_history, 'stage_number': stage_number})
conversation_history += "이우선: " + response + "\n"



> Entering new  chain...
Prompt after formatting:
You are an assistant decide which stage of the conversation to move to or which stage to stay at.
Following '===' is the conversation history. 
Use conversation history to select the next step the agent should take.

Below are the stages of the conversation that the agent can take.
1. Start: This is the first step to take when starting a conversation or responding to a user's first response. Start the conversation by introducing yourself. Be polite and respectful while maintaining a professional tone of conversation.
2. Analyze: When a customer wants a recommendation, run this step before recommendation. This is the step where you identify the user's preferences. Ask enough questions to understand your users' preferences.
3. Recommendation: Once you know the preference of user, you can recommend suitable wines accordingly. Recommendations should be limited to wines in your wine database, and you can use tools for this.
4. After recomm

In [23]:
stage_number = stage_analyzer_chain.run({'conversation_history': conversation_history, 'stage_history': stage_history})
print(f'stage_number: {stage_number}')
stage_history += stage_number if stage_history == "" else ", " + stage_number
response = agent_executor.run({'input':user_response, 'conversation_history': pre_conversation_history, 'stage_number': stage_number})
conversation_history += "이우선: " + response + "\n"



> Entering new  chain...
Prompt after formatting:
You are an assistant decide which stage of the conversation to move to or which stage to stay at.
Following '===' is the conversation history. 
Use conversation history to select the next step the agent should take.

Below are the stages of the conversation that the agent can take.
1. Start: This is the first step to take when starting a conversation or responding to a user's first response. Start the conversation by introducing yourself. Be polite and respectful while maintaining a professional tone of conversation.
2. Analyze: When a customer wants a recommendation, run this step before recommendation. This is the step where you identify the user's preferences. Ask enough questions to understand your users' preferences.
3. Recommendation: Once you know the preference of user, you can recommend suitable wines accordingly. Recommendations should be limited to wines in your wine database, and you can use tools for this.
4. After recomm

ValueError: Could not parse LLM output: `안녕하세요! 와인에 대해 궁금한 점이 있으신가요? <END_OF_TURN>`

In [ ]:
user_response = input("User: ")
pre_conversation_history = conversation_history
conversation_history += f"User: {user_response} <END_OF_TURN>\n"

In [ ]:
for i in conversation_history.split('\n'):
    print(i)

이우선: 안녕하세요! 저는 이우선이라고 합니다. 와인에 대해 궁금한 점이 있으시면 언제든지 물어보세요! 😊<END_OF_TURN>
User: 와인 추천해주세요 <END_OF_TURN>
이우선: 좋아요! 와인 추천을 위해 몇 가지 질문을 드리겠습니다. 먼저, 어떤 종류의 와인을 선호하시나요? (레드, 화이트, 로제, 스파클링 중 선택해주세요) 😊<END_OF_TURN>
User: 레드 와인이요 <END_OF_TURN>
이우선: 좋아요, 레드 와인을 좋아하시군요! 와인의 바디감을 어떻게 선호하시나요? (1: 가벼운 ~ 5: 무거운) 😊<END_OF_TURN>
User: 3정도 <END_OF_TURN>
이우선: 알겠습니다, 바디감이 3 정도인 레드 와인을 찾아볼게요. 가격대는 어느 정도로 생각하고 계신가요? 😊<END_OF_TURN>
User: 10만원 정도 <END_OF_TURN>
이우선: 10만원 이하의 가격대에서 바디감이 3 정도인 레드 와인을 찾았습니다. 다음 와인들을 추천해 드립니다:
1. 비냐 조잘 쿠아트로 델 쿠아트로 (가격: 60,000원, 평점: 3.8) [상세보기](https://www.winenara.com/shop/product/product_view?product_cd=03Q793)
2. 펫저 카베르네소비뇽 (가격: 21,900원, 평점: 3.5) [상세보기](https://www.winenara.com/shop/product/product_view?product_cd=03M899)
3. 비냐 조잘 그라시아노 (가격: 19,000원, 평점: 3.6) [상세보기](https://www.winenara.com/shop/product/product_view?product_cd=03Q791)

이 중에서 마음에 드는 와인이 있으신가요? 😊<END_OF_TURN>
User: 3이 좋을 것 같네요 <END_OF_TURN>
이우선: 좋아요! 비냐 조잘 그라시아노를 선택하셨군요. 구매하시려면 [여기](https://www.winenara.com/shop/p

### Gradio

간단하게 웹 구성을 테스트하는 gradio이다. 개선해야할 점이 많지만 맛보기로 올려보았다.

In [24]:
import gradio as gr

c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
user_response, stage_history, conversation_history, pre_conversation_history = "", "", """""", """"""
 
with gr.Blocks(css='#chatbot .overflow-y-auto{height:750px}') as demo:
    user_response = ""
    conversation_history, pre_conversation_history = """""", """"""
    stage_history = ""
 
    with gr.Row():
        gr.HTML("""<div style="text-align: center; max-width: 500px; margin: 0 auto;">
            <div>
                <h1>ChatWine</h1>
            </div>
            <p style="margin-bottom: 10px; font-size: 94%">
                LinkedIn <a href="https://www.linkedin.com/company/audrey-ai/about/">Audrey.ai</a>
            </p>
        </div>""")
 
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    submit_btn = gr.Button("전송")
    clear_btn = gr.ClearButton([msg, chatbot])

    def answer(user_response, chat_history):
        global conversation_history, pre_conversation_history, stage_history, answer_token, count
        answer_token, count = '', False
        pre_conversation_history = conversation_history
        conversation_history += f"User: {user_response} <END_OF_TURN>\n"
        stage_number = stage_analyzer_chain.run({'conversation_history': conversation_history, 'stage_history': stage_history})
        stage_number = stage_number[-1]
        stage_history += stage_number if stage_history == "" else ", " + stage_number
        print(stage_history)
        response = agent_executor.run({'input':user_response, 'conversation_history': pre_conversation_history, 'stage_number': stage_number})
        conversation_history += "이우선: " + response + "\n"
        response = response.split('<END_OF_TURN>')[0]
        chat_history.append((user_response, response))
 
        return "", chat_history
    
    def user(user_message, history):
        return gr.update(value="", interactive=False), history + [[user_message, None]]

    def clear(*args):
        global conversation_history, pre_conversation_history, stage_history, answer_token
        answer_token = ''
        conversation_history, pre_conversation_history, stage_history = """""", """""", ""


    clear_btn.click(fn=clear)

    submit_btn.click(answer, [msg, chatbot], [msg, chatbot])
    msg.submit(answer, [msg, chatbot], [msg, chatbot])

In [26]:
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4ba06ecba882ba42e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new  chain...
Prompt after formatting:
You are an assistant decide which stage of the conversation to move to or which stage to stay at.
Following '===' is the conversation history. 
Use conversation history to select the next step the agent should take.

Below are the stages of the conversation that the agent can take.
1. Start: This is the first step to take when starting a conversation or responding to a user's first response. Start the conversation by introducing yourself. Be polite and respectful while maintaining a professional tone of conversation.
2. Analyze: When a customer wants a recommendation, run this step before recommendation. This is the step where you identify the user's preferences. Ask enough questions to understand your users' preferences.
3. Recommendation: Once you know the preference of user, you can recommend suitable wines accordingly. Recommendations should be limited to wines in your wine database, and you can use tools for this.
4. After recomm

Traceback (most recent call last):
  File "c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\gradio\routes.py", line 437, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\gradio\blocks.py", line 1352, in process_api
    result = await self.call_function(
  File "c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\gradio\blocks.py", line 1077, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\anyio\_backends\_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "c:\Users\chois\Desktop\chatwine\.venv\lib\site-packages\anyio\_backends\_asyncio.py", line 807, in run
    result = context.run(func, *args)
  File "C:\Users\chois\AppDat



> Entering new  chain...
Prompt after formatting:
You are an assistant decide which stage of the conversation to move to or which stage to stay at.
Following '===' is the conversation history. 
Use conversation history to select the next step the agent should take.

Below are the stages of the conversation that the agent can take.
1. Start: This is the first step to take when starting a conversation or responding to a user's first response. Start the conversation by introducing yourself. Be polite and respectful while maintaining a professional tone of conversation.
2. Analyze: When a customer wants a recommendation, run this step before recommendation. This is the step where you identify the user's preferences. Ask enough questions to understand your users' preferences.
3. Recommendation: Once you know the preference of user, you can recommend suitable wines accordingly. Recommendations should be limited to wines in your wine database, and you can use tools for this.
4. After recomm